

```

```

# Downloading dataset and required modules

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chithrabhanuaalla","key":"84e791c8405fa0e2d14e9a93bd03622e"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets list

ref                                                 title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023               Data Science Salaries 2023 💸                 25KB  2023-04-13 09:55:16           8731        234  1.0              
salvatorerastelli/spotify-and-youtube               Spotify and Youtube                           9MB  2023-03-20 15:43:25          12968        428  1.0              
iammustafatz/diabetes-prediction-dataset            Diabetes prediction dataset                 734KB  2023-04-08 06:11:45           4567         67  1.0              
shawkyelgendy/furniture-price-prediction            Furniture Price Prediction                   81KB  2023-04-17 09:35:17           1265         30  1.0       

In [7]:
!kaggle competitions download -c movie-review-sentiment-analysis-kernels-only

  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 160MB/s]


In [8]:
! mkdir train

In [9]:
! unzip movie-review-sentiment-analysis-kernels-only.zip -d train

Archive:  movie-review-sentiment-analysis-kernels-only.zip
  inflating: train/sampleSubmission.csv  
  inflating: train/test.tsv.zip      
  inflating: train/train.tsv.zip     


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install scikit-learn==0.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [11]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 k

In [12]:
import nltk

In [13]:
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
!ls

drive	     movie-review-sentiment-analysis-kernels-only.zip  train
kaggle.json  sample_data


# Importing modules and reading dataset

In [15]:
import numpy as np # linear algebra
import pandas as pd 
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot as plt
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
import gradio as gr

In [16]:
data = pd.read_csv("/content/train/train.tsv.zip", sep = '\t')

In [17]:
data.shape

(156060, 4)

In [18]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [19]:
phrases = data.iloc[:,2].values
sentiments = data.iloc[:,3].values

In [20]:
print(phrases[0])

A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .


# Cleaning of data

In [21]:
def removeAlphaNumeric(phrase):
    return re.sub("\S*\d\S*", "", phrase).strip()

In [22]:
def removeApostrophe(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"ain't", "ain't", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [23]:
def removeSpecialChar(phrase):
    return re.sub("[^a-zA-Z]", " ", phrase)

In [24]:
def removeStopWords(phrase):
    phrase = phrase.split()
    op_phrase = ""
    stop_words = text.ENGLISH_STOP_WORDS
    for word in phrase:
        if word not in stop_words:
            op_phrase += (word + " ")
    return op_phrase.strip()

In [25]:
def wordLemmatizer(phrase):
    wnl = WordNetLemmatizer()
    return wnl.lemmatize(phrase)

In [26]:
def clean(phrase):
    phrase = removeAlphaNumeric(phrase)
    phrase = phrase.lower()
    phrase = removeApostrophe(phrase)
    phrase = removeSpecialChar(phrase)
    phrase = wordLemmatizer(phrase)
    phrase = removeStopWords(phrase)
    return phrase

In [27]:
#cleaning the data
for i in range(len(phrases)):
    phrases[i] = clean(phrases[i])

In [28]:
print(phrases[0])

series escapades demonstrating adage good goose good gander occasionally amuses amounts story



# Vectorization and splitting

In [29]:
#count_vectorizer = CountVectorizer(max_features = 1000)
#X = count_vectorizer.fit_transform(phrases).toarray()

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 4000)#5500 for bnb
X = vectorizer.fit_transform(phrases).toarray()

In [31]:

X_train, X_test, y_train, y_test = train_test_split(X, sentiments, test_size = 0.20, random_state = 1)

# Training 

Naive Bayes-Gaussian Classifier

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred  = gnb.predict(X_test)

In [ ]:
a = accuracy_score(y_pred, y_test)
print(str('{:04.2f}'.format(a*100))+'%')

12.95%


Naive Bayes-Multinomial Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)

In [ ]:
a = accuracy_score(y_pred, y_test)
print(str('{:04.2f}'.format(a*100))+'%')

58.16%


Naive Bayes-Bernoulli Classifier

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
pred = bnb.predict(X_test)

In [ ]:
b = accuracy_score(pred, y_test)
print(str('{:04.2f}'.format(b*100))+'%')

59.90%


Linear SVC

In [32]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
l_pred = lsvc.predict(X_test)

In [33]:
c = accuracy_score(l_pred, y_test)
print(str('{:04.2f}'.format(c*100))+'%')

61.42%


Logistic Regression

In [ ]:

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

KeyboardInterrupt: ignored

In [ ]:
c = accuracy_score(y_pred, y_test)
print(str('{:04.2f}'.format(c*100))+'%')

# Deployment

In [34]:
def statement(num):
    dict1 = {0: "Negative", 1: "Somewhat Negative", 2: "Neutral", 3: "Somewhat Positive", 4: "Positive"}
    return dict1[num[0]]

In [35]:
def start(review):
    review = clean(review)
    r = (vectorizer.transform([review]).toarray())
    return (statement(lsvc.predict(r)))

In [36]:
print(start("A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story ."))

Somewhat Positive


In [37]:
title = "Moview Review Sentiment Analysis"
description="""
<center>
Predict the sentiment of a movie review
</center>
"""
css = ".gradio-container {background: url('https://png.pngtree.com/background/20220730/original/pngtree-movie-poster-with-film-reel-picture-image_1881622.jpg'); height: 100%; background-position: center; background-repeat: no-repeat; background-size: cover;background-attachment: fixed;}"

In [38]:
outputs = gr.outputs.Textbox()
face = gr.Interface(fn=start, inputs="text", outputs=outputs, title = title, description=description, css=css)
face.launch()

/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>